In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Grab, Set up and Create data

In [ ]:
import numpy as np
import csv
import os.path
import json

#https://stackoverflow.com/questions/26414913/normalize-columns-of-pandas-data-frame
def normalize(column):
    max_value = column.max()
    min_value = column.min()
    result = (column - min_value) / (max_value - min_value)
    
    return result

def readData(data_file):
    raw_data = open(data_file,'rt')
    data = pd.read_csv(raw_data)
    data = data.replace(np.nan,"")
    
    return data

In [ ]:
SPOTIFY_SONG_DATA = "../input/spotify-dataset-19212020-160k-tracks/data_o.csv"

In [ ]:
#TARGET_SONG_ATTRIBUTES = ["acousticness","danceability","energy","instrumentalness","liveness","key","loudness","speechiness","tempo","valence"]
TARGET_SONG_ATTRIBUTES = ["acousticness","danceability","instrumentalness","liveness","speechiness","tempo","valence"]

In [ ]:
SONGS = readData(SPOTIFY_SONG_DATA)
SONGS = SONGS.drop_duplicates(subset=['artists','name'],keep='first')
print(SONGS.shape)

SONGS.head()

In [ ]:
song_data = SONGS[TARGET_SONG_ATTRIBUTES]
song_name = SONGS[['name','artists']]


In [ ]:
for attr in TARGET_SONG_ATTRIBUTES:
    song_data[attr] = normalize(SONGS[attr])
    
song_data

In [ ]:
song_name

## Create training data
data will be song [attributes,recommended song1, recommended song2, recommended song3, recommended song4, recommended song5]

attributes consist of ["acousticness","danceability","instrumentalness","liveness","key","loudness","speechiness","tempo","valence"]

the recommended songs will be the target fields and be represented with their one hot encoded representation


In [ ]:
#Create one hot encoding 
#Encode uses ["name of song","['artist']"] as key
#Decode uses a number as key
ENCODER = dict()
DECODER = dict()

for i,song in enumerate(song_name.values):
    if not (song[0],song[1]) in ENCODER:
        ENCODER[song[0],song[1]] = i
        DECODER[i] = song

In [ ]:
def recommendSongs(song):
    global song_name,song_data
    
    RECOMMENDED_SONGS = []
    distances = []
    
    for pos,targ_song in enumerate(song_data.values):
        distances.append((np.linalg.norm(song - targ_song),pos))
    
    distances.sort(key=lambda x:x[0])
    AMTOFRECOMMENDEDSONGS = 5
    for x in range(AMTOFRECOMMENDEDSONGS):
        RECOMMENDED_SONGS.append(ENCODER[song_name.values[distances[x+1][1]][0],song_name.values[distances[x+1][1]][1]])

    return RECOMMENDED_SONGS

In [ ]:
ENCODER [('Let Me Love You (Until You Learn To Love Yourself)',"['Ne-Yo']")]

In [ ]:
print(SONGS.values[17184])
test = song_data.values[17184]
test

In [ ]:
song_name.values[17184]

In [ ]:
recommend = recommendSongs(test)
for song in recommend:
    print(SONGS.values[song])

In [ ]:
"""
%%time
#Get the recommended list for all the songs
#Not optimized- will take a long time

#Training data is [input song, recommended song1,recommended song2,recommended song3,recommended song4,recommended song5]
TRAINING_DATA = []

for i,input_song in enumerate(song_data.values):
    temp = recommendSongs(input_song)
    TRAINING_DATA.append(temp.insert(0,song_name.values[i]))
   
"""   